### Import libraries

#### Run the same model and add the correctly labeled tweets based on the prediction on the fourth model
- I used KNN to predict the fourth model. Out of the 75 test data, 43 were correct ~ 57%.

In [2]:
import numpy as np
import pandas as pd
import scipy as sp   
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.cross_validation import train_test_split #how does this work?
from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
neighbors = 5
from sklearn.naive_bayes import MultinomialNB
from nltk.classify import NaiveBayesClassifier  #how does this work?

from sklearn.model_selection import cross_val_score as cvs

/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [51]:
kanye2 = pd.read_csv('kanyewest4.csv')
kanye2.head()

,text,label
0,RT @WSHHRAP: Rappers with the most Billboard H...,1.0
1,"Kanye and Cudi merge, together they rule as Kanye",1.0
2,RT @RappersSaid: Rappers with the most Billboa...,1.0
3,RT @c_drew_: Kanye shaped like he played barit...,1.0
4,RT @Bubbabue: @kanyewest Your album #ye is an ...,1.0


In [52]:
kanye2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 2 columns):
text     2064 non-null object
label    589 non-null float64
dtypes: float64(1), object(1)
memory usage: 32.3+ KB


In [56]:
df = kanye2.sort_values(by = 'label', ascending=True)
df1 = df.reset_index()
df1[588:].head()

,index,text,label
588,588,RT @mastaace: Nobody. https://t.co/QUofmPdi5P,1.0
589,589,I need someone who can edit to put Bebe Zahara...,NaN
590,590,@BHOPDaMaestro by Lil Baby ft. Drake &amp; Kan...,NaN
591,591,I’ve never heard Kanye West talk until celebri...,NaN
592,592,Someone stop this guy https://t.co/bEXdtJViQR,NaN


In [57]:
kanye2train = df1[(df1['label'] ==0)|(df1['label']==1)]
kanye2train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 589 entries, 0 to 588
Data columns (total 3 columns):
index    589 non-null int64
text     589 non-null object
label    589 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 18.4+ KB


In [58]:
kanye2train.head()

,index,text,label
0,190,Cudi's album would've way better if Kanye shut...,0.0
1,158,RT @Pink: Kanye west is the biggest piece of s...,0.0
2,159,RT @Pink: Kanye west is the biggest piece of s...,0.0
3,160,kanye west has lost his mind https://t.co/fnPQ...,0.0
4,161,"RT @RealCandaceO: The mob is loud, but the maj...",0.0


In [94]:
kanye2test = df1[df1['label'].isnull()][75:150]
kanye2test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 664 to 738
Data columns (total 3 columns):
index    75 non-null int64
text     75 non-null object
label    0 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ KB


In [95]:
kanye2test.head()

,index,text,label
664,664,Cuz he’s an anti black dick head. Next. https:...,NaN
665,665,Demeter is rarely listening to Kanye West or l...,NaN
666,666,RT @BET: Is Kanye West canceled? Social media ...,NaN
667,667,@BigPatDidThat Talking about Kanye don’t go gh...,NaN
668,668,RT @stankymeat: kanye: aight cudi u ready to r...,NaN


In [96]:
text = kanye2train['text']
print(type(text))
text.head()

<class 'pandas.core.series.Series'>


0    Cudi's album would've way better if Kanye shut...
1    RT @Pink: Kanye west is the biggest piece of s...
2    RT @Pink: Kanye west is the biggest piece of s...
3    kanye west has lost his mind https://t.co/fnPQ...
4    RT @RealCandaceO: The mob is loud, but the maj...
Name: text, dtype: object

In [97]:
toadd = kanye2test['text']
toadd

664    Cuz he’s an anti black dick head. Next. https:...
665    Demeter is rarely listening to Kanye West or l...
666    RT @BET: Is Kanye West canceled? Social media ...
667    @BigPatDidThat Talking about Kanye don’t go gh...
668    RT @stankymeat: kanye: aight cudi u ready to r...
669    @oliverNOtwist JPrince did that to save your G...
670    RT @nastytraka: @BigPatDidThat Talking about K...
671    “Are you voting for Trump/Kanye in the 2020 pr...
672    RT @TheFashionLaw: 21 rape allegations later, ...
673    RT @FoxNews: Kanye West's 'Ye' hits No. 1 on B...
674    @ibarguen @Joi @dkroy @knightfdn @BillClinton ...
675    @Complex Don't u mean Drakes ghostwriter recei...
676    RT @Masuta1019: And them hoes call... https://...
677    @OzLady41563 @VancityReynolds There’s a differ...
678        @jweibel24 Back when I thought Kanye was Kool
679    @WashTimes @realDonaldTrump - Please pick Kany...
680    RT @iamferd97: Man Chance dropped Acid Rap, Ky...
681    RT @balmainbratz: see ho

### Let's try TF-IDF

In [98]:
tfidf = text.append(toadd, ignore_index = True)
tfidf.head()

0    Cudi's album would've way better if Kanye shut...
1    RT @Pink: Kanye west is the biggest piece of s...
2    RT @Pink: Kanye west is the biggest piece of s...
3    kanye west has lost his mind https://t.co/fnPQ...
4    RT @RealCandaceO: The mob is loud, but the maj...
Name: text, dtype: object

In [99]:
vect = TfidfVectorizer(stop_words='english')

In [100]:
tf = pd.DataFrame(vect.fit_transform(tfidf).toarray(), columns=vect.get_feature_names())

In [101]:
tf.head()

,00s,07phill,0jujild7px,10,100,1063atl,10s,11,12,135,...,yp6zcirlxq,ywltohwavw,yxotobksfb,z006mq9emd,zmh9qhh6xk,znu3q5mi58,zopzravtn7,zrhedjtc5k,zth5gym7xn,zuqwdrzfdd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Columns: 1600 entries, 00s to zuqwdrzfdd
dtypes: float64(1600)
memory usage: 8.1 MB


In [103]:
reset = kanye2train['label'].reset_index()
del reset ['index']
reset.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [104]:
kanye = pd.concat([tf,reset], axis = 1,ignore_index= False)
kanye['label'][588:].head()

588    1.0
589    NaN
590    NaN
591    NaN
592    NaN
Name: label, dtype: float64

In [105]:
edittrain = kanye[:589]
edittest = kanye[589:]
droplabel = edittest.drop('label', axis = 1)

In [106]:
edittrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Columns: 1601 entries, 00s to label
dtypes: float64(1601)
memory usage: 7.2 MB


In [107]:
edittest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 589 to 663
Columns: 1601 entries, 00s to label
dtypes: float64(1601)
memory usage: 938.2 KB


In [108]:
x = edittrain.drop('label', axis = 1)
y = edittrain['label']

In [109]:
xtrain, xtest, ytrain, ytest = tts(x,y, random_state = 0)

In [110]:
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
g = GradientBoostingClassifier()
k = KNeighborsClassifier()
nb = MultinomialNB()
vc = VotingClassifier(estimators = [('l',l),('d',d),('r',r),('g',g),('k',k),('nb',nb)],
                      voting = 'hard', n_jobs = 5)

In [111]:
def classifier(ye):
    print(ye)
    clf = ye.fit(xtrain,ytrain)
    clf.predict(droplabel)
    print('Accuracy of train: ', ye.score(xtrain,ytrain))
    print('Accuracy of test: ', ye.score(xtest,ytest))
    score = cvs(ye,xtest,ytest,cv=5, scoring='accuracy')
    print('cross val score: ', score.mean())

In [112]:
classifier(LogisticRegression())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy of train:  0.9002267573696145
Accuracy of test:  0.8918918918918919
cross val score:  0.8370114942528735


In [113]:
classifier(DecisionTreeClassifier())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Accuracy of train:  1.0
Accuracy of test:  0.8918918918918919
cross val score:  0.8436781609195402


In [114]:
classifier(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy of train:  0.9909297052154195
Accuracy of test:  0.9054054054054054
cross val score:  0.8705747126436781


In [115]:
classifier(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
Accuracy of train:  0.9795918367346939
Accuracy of test:  0.9054054054054054
cross val score:  0.8636781609195403


In [116]:
classifier(KNeighborsClassifier())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Accuracy of train:  0.9342403628117913
Accuracy of test:  0.9121621621621622
cross val score:  0.8501149425287355


In [117]:
classifier(MultinomialNB())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Accuracy of train:  0.9750566893424036
Accuracy of test:  0.9256756756756757
cross val score:  0.8570114942528736


In [118]:
classifier(vc)

VotingClassifier(estimators=[('l', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('d', DecisionT...
           weights='uniform')), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=5, voting='hard', weights=None)


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy of train:  0.9931972789115646
Accuracy of test:  0.9054054054054054


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site

cross val score:  0.8703448275862069


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Extracting Ypred

In [128]:
knn = k.fit(xtrain,ytrain)
ypred = knn.predict(droplabel)
ypred

array([1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0.])

In [129]:
len(ypred)

75

In [130]:
len(toadd)

75

In [131]:
results = pd.DataFrame(columns=['text','label'])
results['text'] = toadd
results['label'] = ypred
results.to_csv('kanye.csv')